In [1]:
import tensorflow as tf
import numpy as np
from __future__ import print_function

In [2]:
class HMM(object):

    """
    A class for Hidden Markov Models.

    The model attributes are:
    - K :: the number of states
    - P :: the K by K transition matrix (from state i to state j,
        (i, j) in [1..K])
    - p0 :: the initial distribution (defaults to starting in state 0)
    """

    def __init__(self, P, p0=None):
        self.K = P.shape[0]

        self.P = P
        self.logP = np.log(self.P)

        if p0 is None:
            self.p0 = np.ones(self.K)
            self.p0 /= sum(self.p0)
        elif len(p0) != self.K:
            raise ValueError(
                'dimensions of p0 {} must match P[0] {}'.format(
                    p0.shape, P.shape[0]))
        else:
            self.p0 = p0
        self.logp0 = np.log(self.p0)

    def forward_backward(self, y):
        """
        runs forward backward algorithm on state probabilities y

        Arguments
        ---------
        y : np.array : shape (T, K) where T is number of timesteps and
            K is the number of states

        Returns
        -------
        (posterior, forward, backward)
        posterior : list of length T of tensorflow graph nodes representing
            the posterior probability of each state at each time step
        forward : list of length T of tensorflow graph nodes representing
            the forward probability of each state at each time step
        backward : list of length T of tensorflow graph nodes representing
            the backward probability of each state at each time step
        """
        # set up
        nT = y.shape[0]

        posterior = np.zeros((nT, self.K))
        forward = []
        backward = np.zeros((nT + 1, self.K))

        # forward pass
        forward.append(
            tf.ones((1, self.K), dtype=tf.float64) * (1.0 / self.K)
        )
        for t in range(nT):
            # NOTE: np.matrix expands forward[t, :] into 2d and causes * to be
            # matrix multiplies instead of element wise that an array would be
            tmp = tf.mul(
                tf.matmul(forward[t], self.P),
                y[t]
            )

            forward.append(tmp / tf.reduce_sum(tmp))

        # backward pass
        backward = [None] * (nT + 1)
        backward[-1] = tf.ones((1, self.K), dtype=tf.float64) * (1.0 / self.K)
        for t in range(nT, 0, -1):
            tmp = tf.transpose(
                tf.matmul(
                    tf.matmul(self.P, tf.diag(y[t - 1])),
                    tf.transpose(backward[t])
                )
            )
            backward[t - 1] = tmp / tf.reduce_sum(tmp)

        # remove initial/final probabilities
        forward = forward[1:]
        backward = backward[:-1]

        # combine and normalize
        posterior = [f * b for f, b in zip(forward, backward)]
        posterior = [p / tf.reduce_sum(p) for p in posterior]

        return posterior, forward, backward

    def _viterbi_partial_forward(self, scores):
        # first convert scores into shape [K, 1]
        # then concatenate K of them into shape [K, K]
        expanded_scores = tf.concat(
            1, [tf.expand_dims(scores, 1)] * self.K
        )
        return expanded_scores + self.logP

    def viterbi_decode(self, y, nT):
        """
        Runs viterbi decode on state probabilies y.

        Arguments
        ---------
        y : np.array : shape (T, K) where T is number of timesteps and
            K is the number of states
        nT : int : number of timesteps in y

        Returns
        -------
        (s, pathScores)
        s : list of length T of tensorflow ints : represents the most likely
            state at each time step.
        pathScores : list of length T of tensorflow tensor of length K
            each value at (t, k) is the log likliehood score in state k at
            time t.  sum(pathScores[t, :]) will not necessary == 1
        """

        # pathStates and pathScores wil be of type tf.Tensor.  They
        # are lists since tensorflow doesn't allow indexing, and the
        # list and order are only really necessary to build the unrolled
        # graph.  We never do any computation across all of time at once
        pathStates = []
        pathScores = []

        # initialize
        pathStates.append(None)
        pathScores.append(self.logp0 + np.log(y[0]))

        for t, yy in enumerate(y[1:]):
            # propagate forward
            tmpMat = self._viterbi_partial_forward(pathScores[t])

            # the inferred state
            pathStates.append(tf.argmax(tmpMat, 0))
            pathScores.append(tf.reduce_max(tmpMat, 0) + np.log(yy))

        # now backtrack viterbi to find states
        s = [0] * nT
        s[-1] = tf.argmax(pathScores[-1], 0)
        for t in range(nT - 1, 0, -1):
            s[t - 1] = tf.gather(pathStates[t], s[t])

        return s, pathScores

In [3]:
def latch_P():
    P = np.array([[0.5, 0.5], [0.0, 1.0]])
    # P = np.array([[0.5, 0.5], [0.5, 0.5]])
    # P = np.array([[0.5, 0.5], [0.0000000001, 0.9999999999]])
    # P = np.array([[0.5, 0.5], [1e-50, 1 - 1e-50]])

    for i in range(2):
        for j in range(2):
            print('from', i, 'to', j, P[i, j])
    return P

def fair_P():
    return np.array([[0.5, 0.5], [0.5, 0.5]])

In [4]:
fair_P()

array([[ 0.5,  0.5],
       [ 0.5,  0.5]])

In [5]:
def hmm_tf_fair(fair_P):
    return HMM(fair_P)

def hmm_tf_latch(latch_P):
    return HMM(latch_P)

In [6]:
def lik(y):
    """ given 1d vector of likliehoods length N, return matrix with
    shape (N, 2) where (N, 0) is 1 - y and (N, 1) is y.

    This makes it easy to convert a time series of probabilities
    into 2 states, off/on, for a simple HMM.
    """

    liklihood = np.array([y, y], float).T
    liklihood[:, 0] = 1 - liklihood[:, 0]
    return liklihood

In [7]:
def test_hmm_tf_fair_forward_backward(hmm_tf_fair):
    y = lik(np.array([0, 0, 1, 1]))

    g_posterior, _, _ = hmm_tf_fair.forward_backward(y)
    tf_posterior = np.concatenate(tf.Session().run(g_posterior))

    print('tf_posterior: \n', tf_posterior)

In [8]:
test_hmm_tf_fair_forward_backward(hmm_tf_fair(fair_P()))

tf_posterior: 
 [[ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]]


In [9]:
def test_hmm_fair_forward_backward(hmm_fair):
    y = lik(np.array([0, 0, 1, 1]))

    posterior, f, b = hmm_fair.forward_backward(y)

    # if P is filled with 0.5, the only thing that matters is the emission
    # liklihood.  assert that the posterior is = the liklihood of y
    for i, yi in enumerate(y):
        liklihood = yi / np.sum(yi)
        assert np.isclose(posterior[i, :], liklihood).all()

    # assert that posterior for any given t sums to 1
    assert np.isclose(np.sum(posterior, 1), 1).all()

In [10]:
def test_hmm_latch_two_step_no_noise(hmm_latch):
    for i in range(2):
        for j in range(2):
            y = [i, i, j, j]
            # y = [i, j]

            if i == 1 and j == 0:
                continue

            print('*'*80)
            print(y)
            states, scores = hmm_latch.viterbi_decode(lik(y))

            assert all(states == y)

In [11]:
def test_hmm_tf_partial_forward(hmm_tf_latch):
    scoress = [
        np.log(np.array([0, 1])),
        np.log(np.array([1, 0])),
        np.log(np.array([0.25, 0.75])),
        np.log(np.array([0.5, 0.5])),
    ]

    for scores in scoress:
        tf_ret = tf.Session().run(
            hmm_tf_latch._viterbi_partial_forward(scores))
        print(tf_ret)


In [12]:
test_hmm_tf_partial_forward(hmm_tf_latch(latch_P()))

from 0 to 0 0.5
from 0 to 1 0.5
from 1 to 0 0.0
from 1 to 1 1.0
[[-inf -inf]
 [-inf   0.]]
[[-0.69314718 -0.69314718]
 [       -inf        -inf]]


/Users/marvinbertin/anaconda/envs/TensorFlow-env/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: divide by zero encountered in log
/Users/marvinbertin/anaconda/envs/TensorFlow-env/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/Users/marvinbertin/anaconda/envs/TensorFlow-env/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


[[-2.07944154 -2.07944154]
 [       -inf -0.28768207]]
[[-1.38629436 -1.38629436]
 [       -inf -0.69314718]]


In [13]:
def test_hmm_tf_viterbi_decode(hmm_tf_latch):
    ys = [
        lik(np.array([0, 0])),
        lik(np.array([1, 1])),
        lik(np.array([0, 1])),
        lik(np.array([0, 0.25, 0.5, 0.75, 1])),
    ]

    for y in ys:
        print(y)

        tf_s_graph, tf_scores_graph = hmm_tf_latch.viterbi_decode(y, len(y))
        tf_s = tf.Session().run(tf_s_graph)
        tf_scores = [tf_scores_graph[0]]
        tf_scores.extend([tf.Session().run(g) for g in tf_scores_graph[1:]])
        print(np.array(tf_scores))
        print()

In [14]:
test_hmm_tf_viterbi_decode(hmm_tf_latch(latch_P()))

from 0 to 0 0.5
from 0 to 1 0.5
from 1 to 0 0.0
from 1 to 1 1.0
[[ 1.  0.]
 [ 1.  0.]]
[[-0.69314718        -inf]
 [-1.38629436        -inf]]

[[ 0.  1.]
 [ 0.  1.]]


/Users/marvinbertin/anaconda/envs/TensorFlow-env/lib/python2.7/site-packages/ipykernel/__main__.py:17: RuntimeWarning: divide by zero encountered in log
/Users/marvinbertin/anaconda/envs/TensorFlow-env/lib/python2.7/site-packages/ipykernel/__main__.py:129: RuntimeWarning: divide by zero encountered in log
/Users/marvinbertin/anaconda/envs/TensorFlow-env/lib/python2.7/site-packages/ipykernel/__main__.py:137: RuntimeWarning: divide by zero encountered in log


[[       -inf -0.69314718]
 [       -inf -0.69314718]]

[[ 1.  0.]
 [ 0.  1.]]
[[-0.69314718        -inf]
 [       -inf -1.38629436]]

[[ 1.    0.  ]
 [ 0.75  0.25]
 [ 0.5   0.5 ]
 [ 0.25  0.75]
 [ 0.    1.  ]]
[[-0.69314718        -inf]
 [-1.67397643 -2.77258872]
 [-3.06027079 -3.06027079]
 [-5.13971234 -3.34795287]
 [       -inf -3.34795287]]

